# 아래 변수에 자신의 학번과 이름을 문자열로 저장하세요

In [ ]:
student_number = '20000000'
student_name = '최동혁'

In [ ]:
!pip3 install -r requirements.txt

### 만일, 나눔 폰트를 설치하지 않았다면, 여기서 다운받아서 설치해 주세요.
[네이버 나눔글꼴](https://hangeul.naver.com/font)


In [ ]:
import requests
from lxml import html
import urllib3
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import matplotlib.font_manager as fm
# 나눔 폰트가 없다면 설치하기.
ft = [(f.name, f.fname) for f in fm.fontManager.ttflist if 'Nanum' in f.name][0][0]
plt.rcParams['font.family'] =ft
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


In [ ]:
data = {
    'id': 'kla',
    'topSearchWord': '',
    'pageIndex': '1',
    'initPageUnit': '0',
    'sillokType': 'S',
    'topSearchWord_ime': '',
}

response = requests.post('https://sillok.history.go.kr/search/inspectionMonthList.do', data=data, verify=False)
month_list = [link.split("', ")[0].replace("javascript:search('",'') for link in html.fromstring(response.text).xpath('//*[@id="cont_area"]/div/div[2]/ul[2]/li/ul/li//a/@href')]

In [ ]:
days_list = []
for month in month_list:
    data = {
        'id': month,
        'did': '',
        'level': '3',
    }
    response = requests.post('https://sillok.history.go.kr/search/inspectionDayList.do', data=data, verify=False)
    days = html.fromstring(response.text).xpath('//*[@id="cont_area"]/div[1]/div[3]/div/dl/dd/ul/li/a/@href')
    days = [day.split("',")[0].replace("javascript:searchView('",'').replace("');",'') for day in days]
    days_list.extend(days)

### 문제. day_xpath 변수에 들어갈 적절한 표현을 넣어 주세요.
### 분류에 해당하는 요소 선택하기

In [ ]:
base_url ='https://sillok.history.go.kr/id/'
injong_categories = []
for day_url in days_list:
    response = requests.get(base_url+day_url, verify=False)
    day_xpath = '//*[@id="cont_area"]/div[1]/div[3]/div[1]/div/div/ul[1]/li[2]/div'
    categories = [c.text.split(' / ') for c in html.fromstring(response.text).xpath(day_xpath)]
    injong_categories.extend(categories)

## 아래 코드를 실행시키고, 출력된 결과를 스크린 캡쳐 또는 이미지 파일로 만들어 블랙보드에 제출합니다.

In [ ]:
flat_injong_categories = [item.split("(")[0] for sublist in injong_categories for item in sublist]
cnt = Counter(flat_injong_categories)
df = pd.DataFrame(cnt.most_common(10), columns=['Category', 'Count'])
df.plot(x='Category', y='Count', kind='bar')
plt.title(f'{student_number} {student_name}')